In [108]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import requests
from spotipy.oauth2 import SpotifyOAuth
from tqdm import tqdm
import sys
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import time
from google.cloud import bigquery
import os

import pandas_gbq
%load_ext google.cloud.bigquery

# Set your default project here
pandas_gbq.context.project = 'snappy-boulder-378707'
pandas_gbq.context.dialect = 'standard'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../letsgo-snappy-boulder-378707-4b7d46801fd1.json"

# Construct a BigQuery client object.
client = bigquery.Client()


from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, RobustScaler, OneHotEncoder

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


# Genre

In [127]:
import joblib
import pickle
gb = pickle.load(open('../../Model/gb_0.4343.sav','rb'))
#gb = joblib.load('../../Model/gb_0.4343.sav')

In [156]:
%%bigquery genre
SELECT distinct * FROM snappy-boulder-378707.NewReleases.NewAudioFeatures

Query is running:   0%|          |

Downloading:   0%|          |

In [157]:
dummy_cols = ['key','mode','time_signature']
genre= pd.get_dummies(genre, columns=dummy_cols)
genre.drop(['energy', 'loudness'],axis=1, inplace=True)

In [158]:
features = pd.read_csv("../../Model/Genre_classification_features.csv")

In [159]:
for col in set(features.iloc[:,0].values) - set(genre.iloc[:,1:].columns):
    genre[col] = 0
sc = MinMaxScaler(feature_range = (0,100))
genre_trans = sc.fit_transform(genre.iloc[:,1:])

In [160]:
pred_genre = gb.predict(genre_trans)

In [162]:
genre['Genre'] = pred_genre

In [164]:
genre = genre[['id','Genre']]

In [165]:
genre

,id,Genre
0,7rXQXbK4pQo8DSKlZ9ZSjC,K-Pop
1,2LRqm962By8aNwJX0WxFNG,Folk & Acoustic
2,0Oqxt6JixieLHbwMfnJGWO,Jazz
3,0tYtfBFJLVn2ITE66FbaLT,Indie
4,5ZSqX1dDJ823Dndw94nogY,Dance Electronic
...,...,...
179,1Oi8jp3BvmESVXIPtJ06a5,K-Pop
180,2lxwvwNTCT6T2k9agnujeO,Mandopop
181,6f6jhTzxwvAYhzZrABTO8T,Malay Pop
182,4Fx7GtXOK8k3rxVgKzi6ko,Mandopop


# Popularity

In [143]:
%%bigquery popularity
SELECT distinct s1.id,
s1.explicit,
s1.available_markets,
artist_id,
danceability,
energy,
key,
loudness,
mode,
speechiness,
acousticness,
instrumentalness,
liveness,
valence,
tempo,
duration_ms,
time_signature,
release_date FROM `snappy-boulder-378707.NewReleases.NewTracks`  as s1
inner join `snappy-boulder-378707.NewReleases.NewAudioFeatures`as s2
on s1.id = s2.id
inner join `snappy-boulder-378707.NewReleases.NewAlbums` as s3
on s1.album_id = s3.id

Query is running:   0%|          |

Downloading:   0%|          |

In [139]:
%%bigquery artist
SELECT distinct id, followers, popularity FROM `snappy-boulder-378707.TrackClearInfo.ArtistInfo`

Query is running:   0%|          |

Downloading:   0%|          |

In [144]:
popularity

,id,explicit,available_markets,artist_id,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,release_date
0,4QLqCFAGqvHnjOWCtHTsQL,0,105,[;';7;q;r;E;X;i;L;L;n;W;k;k;Y;H;h;a;d;Z;1;O;i;...,0.576,0.446,6,-6.509,0,0.0427,0.00598,0.000551,0.0899,0.662,91.995,117850,4,2023-04-07
1,7f1mKhZ8s6di51f76Qf03E,0,105,[;';7;q;r;E;X;i;L;L;n;W;k;k;Y;H;h;a;d;Z;1;O;i;...,0.461,0.550,6,-5.439,0,0.0325,0.13700,0.000002,0.1090,0.526,82.991,143687,4,2023-04-07
2,2pqi703QQcp2zsuS3NgCii,0,105,[;';7;q;r;E;X;i;L;L;n;W;k;k;Y;H;h;a;d;Z;1;O;i;...,0.586,0.548,0,-7.453,1,0.0249,0.11400,0.000023,0.2220,0.485,139.997,216852,4,2023-04-07
3,4wHo9RDSY0elxbeiMpYNLN,0,105,[;';7;q;r;E;X;i;L;L;n;W;k;k;Y;H;h;a;d;Z;1;O;i;...,0.560,0.798,7,-5.197,1,0.0357,0.00794,0.793000,0.0907,0.271,112.016,274348,4,2023-04-07
4,1oMceEpywwehJQWKGueTlc,0,105,[;';7;q;r;E;X;i;L;L;n;W;k;k;Y;H;h;a;d;Z;1;O;i;...,0.565,0.694,7,-5.402,1,0.0294,0.00443,0.000004,0.2880,0.402,135.991,226661,4,2023-04-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,1FD5Ls4IazGePpuS5PphMn,0,184,762310PdDnwsDxAQxzQkfX,0.405,0.762,9,-5.826,0,0.0536,0.03230,0.023700,0.0912,0.385,179.875,228474,3,2023-03-24
253,2ARxntdEqBj9MeKFuckr8R,0,184,762310PdDnwsDxAQxzQkfX,0.629,0.774,10,-4.667,0,0.0279,0.00528,0.244000,0.0857,0.406,120.016,237297,4,2023-03-24
254,32IMgcVhysYW12NRvMLp8c,0,182,24V5UY0nChKpnb1TBPJhCw;68WwJXWrpo1yVOOIZjLSeT,0.674,0.742,0,-6.374,1,0.0384,0.01170,0.000003,0.2810,0.604,135.007,186000,4,2023-03-22
255,1ENQZlb9iVUnCoN2QtI77e,0,183,45eNHdiiabvmbp4erw26rg;164Uj4eKjl6zTBKfJLFKKK,0.565,0.697,9,-4.901,1,0.0316,0.32100,0.000000,0.0926,0.343,99.934,177600,4,2023-03-24


In [3]:
%%bigquery tracks
SELECT id, explicit, available_markets,album_id,artist_id FROM snappy-boulder-378707.NewReleases.NewTracks

Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
ct = ColumnTransformer([('minmax', MinMaxScaler(), ['release_date','loudness', 'tempo', 'duration_ms','available_markets','followers','popularity_artist']),
                        ('categorical', OneHotEncoder(), ['key','mode','time_signature','explicit']),
                       ],
                       remainder='passthrough')

ct.fit(X_train)

X_train_preprocessed = pd.DataFrame(ct.transform(X_train).tolist())